In [1]:
from transformers import pipeline
from transformers import AutoTokenizer
import torch
# import numpy as np
import pandas as pd
from transformers import TrainingArguments, Trainer
model="distilbert-base-uncased-finetuned-sst-2-english"
# Load tokenizer for the model

/opt/anaconda3/envs/Stats/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
if torch.backends.mps.is_available():
    device = torch.device("mps")  # Use MPS (Metal GPU)
else:
    device = torch.device("cpu")  # Fallback to CPU

# Check if MPS is available
device = 0 if torch.backends.mps.is_available() else -1
print(f"Using device: {'MPS' if device == 0 else 'CPU'}")

Using device: MPS


In [3]:
sentiment_pipeline = pipeline("sentiment-analysis", model=model, device=device)


In [4]:
import os

# Set the environment variable
os.environ["MKL_SERVICE_FORCE_INTEL"] = "1"


In [5]:
data = ["I love you", "I hate you"]
r = sentiment_pipeline(data)
r

[{'label': 'POSITIVE', 'score': 0.9998656511306763},
 {'label': 'NEGATIVE', 'score': 0.9991129040718079}]

In [6]:
tokenizer = AutoTokenizer.from_pretrained(model)
tokenizer

DistilBertTokenizerFast(name_or_path='distilbert-base-uncased-finetuned-sst-2-english', vocab_size=30522, model_max_length=512, is_fast=True, padding_side='right', truncation_side='right', special_tokens={'unk_token': '[UNK]', 'sep_token': '[SEP]', 'pad_token': '[PAD]', 'cls_token': '[CLS]', 'mask_token': '[MASK]'}, clean_up_tokenization_spaces=True),  added_tokens_decoder={
	0: AddedToken("[PAD]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	100: AddedToken("[UNK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	101: AddedToken("[CLS]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	102: AddedToken("[SEP]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
	103: AddedToken("[MASK]", rstrip=False, lstrip=False, single_word=False, normalized=False, special=True),
}

In [7]:
print(torch.cuda.is_available())  # Returns True if a GPU is available
print(torch.cuda.device_count())  # Number of GPUs


False
0


In [8]:
# from datasets import load_dataset

# # Load dataset from a CSV file
# data_files = {"train": "path_to_train.csv", "validation": "path_to_val.csv"}  # Update paths
# dataset = load_dataset("csv", data_files=data_files)

# # Example structure: {'text': ..., 'label': ...}
# print(dataset)


In [9]:
from utils_classes import load_and_process_comments


train_comments, val_comments, test_comments, test_labels = load_and_process_comments(
    train_path='train',
    batch_size=50,
)
# Flatten train_comments
train_texts = [text for batch in train_comments for text in batch[0]]
train_labels = [label for batch in train_comments for label in batch[1]]

# Flatten test_comments
test_texts = [text for batch in test_comments for text in batch[0]]
test_labels = [label for batch in test_comments for label in batch[1]]


In [10]:
# Tokenize training and test data
train_encodings = tokenizer(train_texts, truncation=True, padding=True, max_length=512)
test_encodings = tokenizer(test_texts, truncation=True, padding=True, max_length=512)


In [11]:
type(train_encodings)


transformers.tokenization_utils_base.BatchEncoding

In [12]:
class CommentsDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])
        return item

# Create dataset objects
train_dataset = CommentsDataset(train_encodings, train_labels)
test_dataset = CommentsDataset(test_encodings, test_labels)

In [13]:
from transformers import AutoModelForSequenceClassification

# Replace "distilbert-base-uncased" with your model's name if needed
model_new = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased", 
    num_labels=2  # Adjust `num_labels` based on your dataset (e.g., binary classification)
)


Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [14]:
training_args = TrainingArguments(
    output_dir="./results",          # Directory to save the model
    eval_strategy="epoch",          # Evaluate after each epoch (updated argument)
    learning_rate=2e-5,             # Learning rate
    per_device_train_batch_size=16, # Batch size for training
    per_device_eval_batch_size=16,  # Batch size for evaluation
    num_train_epochs=3,             # Number of epochs
    weight_decay=0.01,              # Weight decay
    logging_dir="./logs",           # Directory for logs
    logging_steps=10,               # Log every 10 steps
    save_strategy="epoch",          # Save checkpoint each epoch
    load_best_model_at_end=True,    # Load best model at the end of training
)

In [ ]:
trainer = Trainer(
    model=model_new,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=test_dataset,
    tokenizer=tokenizer,
)

trainer.train()

/var/folders/27/h86gj_fd3xg23kmxw_95c6kc0000gn/T/ipykernel_2205/3123840680.py:1: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer.__init__`. Use `processing_class` instead.
  trainer = Trainer(
  0%|          | 10/5625 [00:29<4:04:18,  2.61s/it]

{'loss': 0.6913, 'grad_norm': 1.0757776498794556, 'learning_rate': 1.9964444444444447e-05, 'epoch': 0.01}


  0%|          | 20/5625 [00:54<3:58:54,  2.56s/it]

{'loss': 0.6766, 'grad_norm': 1.3927199840545654, 'learning_rate': 1.992888888888889e-05, 'epoch': 0.01}


  1%|          | 30/5625 [01:19<3:56:47,  2.54s/it]

...

 72%|███████▏  | 4030/5625 [3:07:32<1:14:07,  2.79s/it]

{'loss': 0.103, 'grad_norm': 13.233839988708496, 'learning_rate': 5.671111111111112e-06, 'epoch': 2.15}


 72%|███████▏  | 4040/5625 [3:07:58<1:10:21,  2.66s/it]

{'loss': 0.1833, 'grad_norm': 4.650296211242676, 'learning_rate': 5.635555555555557e-06, 'epoch': 2.15}


 72%|███████▏  | 4049/5625 [3:08:22<1:12:13,  2.75s/it]

In [9]:
import torch

# Check PyTorch version
print("PyTorch version:", torch.__version__)

# Check MPS availability
if torch.backends.mps.is_available():
    print("MPS is available and will be used for acceleration!")
else:
    print("MPS is not available. Running on CPU.")


PyTorch version: 2.2.2
MPS is available and will be used for acceleration!
